# Limpieza inicial de las tablas

In [ ]:
import re
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# ---------------------------
# FUNCIONES AUXILIARES
# ---------------------------
def limpiar_dataframe(df):
    """Estandariza nombres de columnas y reemplaza valores nulos o inválidos."""
    df.columns = [str(col).strip().lower().replace(" ", "_") for col in df.columns]
    df.replace(["s/d", "S/D", "-", "", " "], np.nan, inplace=True)
    return df

def convertir_a_numerico(df, columnas_excluidas=[]):
    for col in df.columns:
        if col not in columnas_excluidas:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# ---------------------------
# CARGA DE DATOS
# ---------------------------

# 1. Focos de calor por jurisdicción y año
archivo = os.path.join("datasets","Cantidad de focos de calor registrados por jurisdicción y año.xlsx")
focos = pd.read_excel(archivo)
focos = limpiar_dataframe(focos)
focos = convertir_a_numerico(focos, columnas_excluidas=["jurisdicción"])

# 2. Incendios por provincia anualizados

archivo = os.path.join("datasets","Incendios por provincia.xlsx")
incendios = pd.read_excel(archivo)
incendios = limpiar_dataframe(incendios)
incendios = convertir_a_numerico(incendios, columnas_excluidas=["jurisdicción", "año"])

# 3. Estadísticas normales (múltiples hojas)
archivo = os.path.join("datasets","Estadísticas normales Datos abiertos 1991-2020- TODAS HOJAS.xlsx")
estadisticas = pd.read_excel(archivo, sheet_name=None)
estadisticas_limpias = {}
for hoja, df in estadisticas.items():
    df = limpiar_dataframe(df)
    df = convertir_a_numerico(df)
    estadisticas_limpias[hoja] = df

# 4. Densidad de población por cuenca

archivo = os.path.join("datasets","Densidad de población por cuenca.xlsx")
poblacion = pd.read_excel(archivo)
poblacion = limpiar_dataframe(poblacion)
poblacion = convertir_a_numerico(poblacion, columnas_excluidas=["provincia"])

# 5. Superficie afectada
archivo = os.path.join("datasets","Superficie afectada por incendios reportados, por jurisdicción.xlsx")
superficie_abs = pd.read_excel(archivo, sheet_name="rii_c_hectareas_incendios_prov_")
superficie_pct = pd.read_excel(archivo, sheet_name="Porcentaje de superficie afecta")

superficie_abs = limpiar_dataframe(superficie_abs)
superficie_abs = convertir_a_numerico(superficie_abs, columnas_excluidas=["jurisdicción"])

superficie_pct = limpiar_dataframe(superficie_pct)
superficie_pct["%_suma_de_hectareas"] = pd.to_numeric(superficie_pct["%_suma_de_hectareas"], errors='coerce')

# 6. Supertabla
archivo = os.path.join("datasets","Supertabla.xlsx")
supertabla = pd.read_excel(archivo, sheet_name=None)
df1 = limpiar_dataframe(supertabla['hoja1'])
df2 = limpiar_dataframe(supertabla['hoja2'])

df1 = convertir_a_numerico(df1, columnas_excluidas=["jurisdicción"])
df2 = convertir_a_numerico(df2, columnas_excluidas=["jurisdicción", "combustible"])

#7. Incendios por mes
archivo = os.path.join("datasets","Cantidad de incendios reportados por mes.xlsx")
incendios_mes = pd.read_excel(archivo)

incendios_mes = limpiar_dataframe(incendios_mes)
incendios_mes = convertir_a_numerico(incendios_mes, columnas_excluidas=["mes"])


#8. Focos por mes por jurisdiccion
archivo = os.path.join("datasets", "cantidad_focos_mes_jurisdiccion.xlsx")
focos_mes = pd.read_excel(archivo)

focos_mes = limpiar_dataframe(focos_mes)
focos_mes = convertir_a_numerico(focos_mes, columnas_excluidas=["jurisdicción"])


# ---------------------------
# REVISIÓN POST-LIMPIEZA
# ---------------------------
print("✔ Limpieza completada.")
print("Dimensiones focos:", focos.shape)
print("Dimensiones incendios:", incendios.shape)
print("Dimensiones población:", poblacion.shape)
print("Dimensiones superficie afectada:", superficie_abs.shape)
print("Dimensiones supertabla_1:", df1.shape)
print("Dimensiones supertabla_2:", df2.shape)
print("Dimensiones incendios por mes:", incendios_mes.shape)
print("Dimensiones Focos por mes:", focos_mes.shape)
incendios_mes


# Analisis exploratorio de los Datos

In [ ]:
# Estilo gráfico y orden de meses
sns.set_theme(style="whitegrid", palette="OrRd")  # estilo limpio
plt.rcParams["figure.figsize"] = (10, 6)

orden_meses = [
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
    'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'
]

### Distribucion de focos de calor por jurisdiccion ( En promedio )

In [ ]:
promedio_anios = focos.columns[-1]
focos_ultimo = focos[["jurisdicción", promedio_anios]].sort_values(by=promedio_anios, ascending=False)

plt.figure()
sns.barplot(data=focos_ultimo, x=promedio_anios, y="jurisdicción", palette="Reds_r")
plt.title(f"Focos de calor por jurisdicción - {promedio_anios}")
plt.xlabel("Cantidad de focos")
plt.ylabel("Jurisdicción")
plt.tight_layout()
plt.show()


###  Evolución temporal de focos de calor a nivel nacional

In [ ]:
# Excluimos la columna "jurisdicción" y la última columna del DataFrame
columnas_a_sumar = focos.columns[1:-1]  # Seleccionamos todas las columnas excepto la primera ("jurisdicción") y la última("Promedio")

# Sumamos únicamente las columnas numéricas de los años
focos_anual = (
    focos[columnas_a_sumar]  # Nos quedamos solo con las columnas correspondientes a años
    .sum()  # Sumamos los valores por cada año
    .reset_index()  # Convertimos el índice a columna para reestructurar el DataFrame
)

# Renombramos las columnas
focos_anual.columns = ["año", "total_focos"]

# Quitamos el prefijo "año_" en caso de que los nombres contengan este prefijo
focos_anual["año"] = focos_anual["año"].str.replace("año_", "").astype(int)

# Visualizamos los resultados con Seaborn
plt.figure()
sns.lineplot(data=focos_anual, x="año", y="total_focos", marker="o", color="orangered")
plt.title("Evolución anual de focos de calor")
plt.xlabel("Año")
plt.ylabel("Total de focos de calor")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Superficie afectada promedio por jurisdicción

In [ ]:
# Verificamos las columnas
columnas_a_promediar = superficie_abs.columns[1:-1]  # Excluimos la primera columna (jurisdicción) y la última (promedio)

# Calculamos de nuevo el promedio de superficie afectada por jurisdicción (si es necesario)
superficie_abs["promedio"] = superficie_abs[columnas_a_promediar].mean(axis=1)

# Seleccionamos la columna 'promedio' para el gráfico
superficie_media = (
    superficie_abs.set_index("jurisdicción")["promedio"]  # Usamos la columna promedio
    .sort_values(ascending=False)  # Ordenamos de mayor a menor
)

# Graficamos
plt.figure(figsize=(10, 6))
sns.barplot(x=superficie_media.values, y=superficie_media.index, palette="YlOrBr")
plt.title("Promedio de superficie afectada por jurisdicción (ha)")
plt.xlabel("Hectáreas promedio")
plt.ylabel("Jurisdicción")
plt.tight_layout()
plt.show()

### Relación entre densidad poblacional y superficie afectada (último año)

In [ ]:
# (Si la columna original se llama 'provincia' en poblacion)
poblacion["jurisdicción"] = poblacion["provincia"].str.strip().str.lower()
superficie_abs["jurisdicción"] = superficie_abs["jurisdicción"].str.strip().str.lower()

df_comb = pd.merge(
    poblacion, 
    superficie_abs[["jurisdicción", superficie_abs.columns[-4]]], 
    on="jurisdicción", 
    how="inner"
)

plt.figure()
sns.scatterplot(data=df_comb, x="prom_densidad_hab_km2", y=superficie_abs.columns[-4], hue="jurisdicción", palette="tab20")
plt.title(f"Relación entre densidad poblacional y superficie afectada ({superficie_abs.columns[-4]})")
plt.xlabel("Densidad poblacional (hab/km²)")
plt.ylabel("Superficie afectada (ha)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## Correlaciones profundas
* Se busca entender si hay relación estadística entre:

    🔥 Focos de calor

    🧯 Incendios reportados

    🌾 Superficie afectada

    👥 Densidad poblacional

### Preparacion de la tabla

In [ ]:
cols_focos = [c for c in focos.columns if c.startswith("año_") or c.isdigit()]
cols_inc = [c for c in incendios.columns if c.startswith("año_") or c.isdigit()]
cols_sup = [c for c in superficie_abs.columns if c.startswith("hectáreas_") or c.isdigit()]
cols_focos_mes = [c for c in focos_mes.columns if c in orden_meses]
cols_inc_mes = [c for c in incendios_mes.columns if c.startswith("año_") or c.isdigit()]

In [ ]:
focos["focos_prom"] = focos[cols_focos].mean(axis=1)
incendios["incendios_prom"] = incendios[cols_inc].mean(axis=1)
superficie_abs = superficie_abs.rename(columns={"promedio": "superficie_afectada_prom"})


### Normalizacion de clave jurisdiccion

In [ ]:
# Diccionario para normalizar nombres de jurisdicciones
normalizacion_jurisdicciones = {
    "cordoba": "córdoba",
    "entre rios": "entre ríos",
    "neuquen": "neuquén",
    "rio negro": "río negro",
    "tucuman": "tucumán",
    "stgo. del estero": "santiago del estero",
    "t. del fuego": "tierra del fuego"
}


In [ ]:
def normalizar_jurisdicciones(df):
    df["jurisdicción"] = df["jurisdicción"].str.strip().str.lower()
    df["jurisdicción"] = df["jurisdicción"].replace(normalizacion_jurisdicciones)
    return df

# Aplicar a todos los DataFrames
focos = normalizar_jurisdicciones(focos)
incendios = normalizar_jurisdicciones(incendios)
superficie_abs = normalizar_jurisdicciones(superficie_abs)
poblacion = normalizar_jurisdicciones(poblacion)


In [ ]:
df_corr = focos[["jurisdicción", "focos_prom"]].merge(
    incendios[["jurisdicción", "incendios_prom"]], on="jurisdicción", how="inner"
).merge(
    superficie_abs[["jurisdicción", "superficie_afectada_prom"]], on="jurisdicción", how="inner"
).merge(
    poblacion[["jurisdicción", "prom_densidad_hab_km2"]], on="jurisdicción", how="left"
)
# Eliminar filas con NaN en variables críticas:
df_corr = df_corr.dropna(subset=["focos_prom", "incendios_prom", "superficie_afectada_prom", "prom_densidad_hab_km2"])


In [ ]:
corr_mat = df_corr.drop(columns="jurisdicción").corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr_mat, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación entre variables anuales por jurisdicción")
plt.tight_layout()
plt.show()


In [ ]:
# Melt con columnas ya filtradas por año (definidas antes)
focos_largo = focos.melt(id_vars="jurisdicción", value_vars=cols_focos, var_name="año", value_name="cantidad_focos")
focos_largo["año"] = focos_largo["año"].apply(lambda x: int(re.search(r"\d{4}", x).group()))

incendios_largo = incendios.melt(id_vars="jurisdicción", value_vars=cols_inc, var_name="año", value_name="cantidad_incendios")
incendios_largo["año"] = incendios_largo["año"].apply(lambda x: int(re.search(r"\d{4}", x).group()))

superficie_largo = superficie_abs.melt(id_vars="jurisdicción", value_vars=cols_sup, var_name="año", value_name="superficie_afectada")
superficie_largo["año"] = superficie_largo["año"].apply(lambda x: int(re.search(r"\d{4}", x).group()))


In [ ]:
df_ja = focos_largo.merge(incendios_largo, on=["jurisdicción","año"], how="inner") \
                   .merge(superficie_largo, on=["jurisdicción","año"], how="inner") \
                   .merge(poblacion[["jurisdicción","prom_densidad_hab_km2"]], on="jurisdicción", how="left")
df_ja = df_ja.dropna(subset=["cantidad_focos","cantidad_incendios","superficie_afectada","prom_densidad_hab_km2"])


In [ ]:
corr_mat2 = df_ja[["cantidad_focos","cantidad_incendios","superficie_afectada","prom_densidad_hab_km2"]].corr()
sns.heatmap(corr_mat2, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación nivel jurisdicción-año")
plt.show()


# Visualización estacional de incendios
* Objetivo: detectar meses con mayor promedio de incendios y patrones de estacionalidad.

### Enfoque Estacional Nacional

In [ ]:
#Convertir a formato largo
df_inc_nac = incendios_mes.melt(id_vars="mes", value_vars=cols_inc_mes, var_name="año", value_name="cantidad")

#Normalizar mes

df_inc_nac["mes"] = df_inc_nac["mes"].str.lower()

In [ ]:
#Promedio mensual nacional
prom_nacional = df_inc_nac.groupby("mes")["cantidad"].mean().reindex(orden_meses).reset_index()

#Grafico de barras
plt.figure(figsize=(10, 5))
sns.barplot(data=prom_nacional, x="mes", y="cantidad")
plt.title("Promedio mensual nacional de incendios (2017–2024)")
plt.xticks(rotation=45)
plt.ylabel("Cantidad promedio")
plt.xlabel("Mes")
plt.tight_layout()
plt.show()

### Enfoque Estacional Provincial (Proxy)

In [ ]:
#Convertir a formato largo
df_focos_largo = focos_mes.melt(id_vars=["jurisdicción","año"], value_vars=cols_focos_mes,var_name="mes",value_name="cantidad_focos")

#Normalizar nombres de provincias 
df_focos_largo = normalizar_jurisdicciones(df_focos_largo)

df_focos_largo["mes"] = df_focos_largo["mes"].str.lower()

In [ ]:
# Promedio mensual por jurisdicción
prom_focos_jur = (
    df_focos_largo.groupby(["jurisdicción", "mes"])["cantidad_focos"]
    .mean()
    .reset_index()
)

# Ordenar meses para visualización
prom_focos_jur["mes"] = pd.Categorical(prom_focos_jur["mes"], categories=orden_meses, ordered=True)
prom_focos_jur = prom_focos_jur.sort_values(["jurisdicción", "mes"])


! Ayuda de interpretacion: Por no contar con el dataset apropiado ( incendios por mes por jurisdiccion) y teniendo en cuenta que la correlacion
de incendios con focos es baja aprox -0.20, este analisis debe ser tratado con cuidado, ya que los valores pueden no ser representativos y solo sirven como una aproximacion.

In [ ]:
# Gráfico múltiple: estacionalidad por provincia (proxy)
g = sns.FacetGrid(prom_focos_jur, col="jurisdicción", col_wrap=4, height=3.5, sharey=False)
g.map(sns.barplot, "mes", "cantidad_focos", order=orden_meses)
g.set_titles("{col_name}")
g.set_xticklabels(rotation=45)
g.figure.suptitle("Estacionalidad de focos de calor por jurisdicción (proxy de incendios)", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Mes con mayor promedio por jurisdicción (10 más altos)
mes_max_focos = (
    prom_focos_jur.loc[prom_focos_jur.groupby("jurisdicción")["cantidad_focos"].idxmax()]
    .sort_values("cantidad_focos", ascending=False)
    .reset_index(drop=True)
)

mes_max_focos.rename(columns={"mes": "mes_pico", "cantidad_focos": "focos_promedio_max"}, inplace=True)
mes_max_focos.head(10)


In [ ]:
# Obtener top 3 meses con más focos promedio por provincia
top3_focos = (
    prom_focos_jur.sort_values(["jurisdicción", "cantidad_focos"], ascending=[True, False])
    .groupby("jurisdicción")
    .head(3)
    .reset_index(drop=True)
)

# Mostrar ejemplo
top3_focos.head(9)


# Preparacion Para Machine Learning